<div class="alert alert-warning">
    
<b>Disclaimer:</b> The main objective of the <i>Jupyter</i> notebooks is to show how to use the models of the <i>QENS library</i> by
    
- building a fitting model: composition of models, convolution with a resolution function
- setting and run the fit   
- extracting and displaying information about the results

These steps have a minimizer-dependent syntax. That's one of the reasons why different minimizers have been used in the notebooks provided as examples.  
But, the initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.
</div>

# Example: Composition of Lorentzian and background model fitted with lmfit

## Table of Contents

- [Introduction](#Introduction)
- [Importing the required libraries](#Importing-the-required-libraries)
- [Plot of the fitting model](#Plot-of-the-fitting-model)
- [Creating the reference data](#Creating-the-reference-data)
- [Setting and fitting](#Setting-and-fitting)  
- [Plotting the results](#Plotting-the-results) 

[Top](#Table-of-Contents)

## Introduction

<div class="alert alert-info">
    
The objective of this notebook is to show how to combine the models of 
the <a href="https://github.com/QENSlibrary/QENSmodels">QENSlibrary</a>. Here, we use the <b>Lorentzian</b> profile and a flat background, created from <b>background_polynomials</b>, to perform some fits.

<a href="https://lmfit.github.io/lmfit-py/">lmfit</a> is used for fitting.
</div>

### Physical units

For information about unit conversion, please refer to the jupyter notebook called `Convert_units.ipynb` in the `tools` folder.

The dictionary of units defined in the cell below specify the units of the refined parameters adapted to the convention used in the experimental datafile.

In [ ]:
# Units of parameters for selected QENS model and experimental data
dict_physical_units = {'omega': "1/ps", 
                       'scale': "unit_of_signal/ps", 
                       'center': "1/ps", 
                       'hwhm': "1/ps",}

[Top](#Table-of-Contents)

## Importing the required librairies

In [ ]:
# import python modules for plotting, fitting
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

# for interactivity (plots, buttons...)
from pandas import DataFrame
import panel.widgets as pnw
import panel as pn
pn.extension()

%matplotlib notebook

In [ ]:
# install QENSmodels (if not already installed)
import pkgutil
import sys

if not pkgutil.find_loader("QENSmodels"):
    buttonY = pnw.Button(name='Yes', button_type='success')
    buttonN = pnw.Button(name='No', button_type='danger')
    choice_installation = panel.Column("Do you want to install the QENSmodels' library?", panel.Row(buttonY, buttonN))
    display(choice_installation)

In [ ]:
if not pkgutil.find_loader("QENSmodels"):
    if buttonY.clicks>0:
        !{sys.executable} -m pip install git+https://github.com/QENSlibrary/QENSmodels#egg=QENSmodels
    elif buttonN.clicks>0:
        print("You will not be able to run some of the remaining parts of this notebook")

In [ ]:
# install lmfit (if not already installed)
if not pkgutil.find_loader("lmfit"):
    lmfitY = pnw.Button(name='Yes', button_type='success')
    lmfitN = pnw.Button(name='No', button_type='danger')
    choice_installation = panel.Column("Do you want to install lmfit?", panel.Row(lmfitY, lmfitN))
    display(choice_installation)

In [ ]:
if not pkgutil.find_loader("lmfit"):
    if lmfitY.clicks>0:
         !{sys.executable} -m pip install lmfit
    elif lmfitN.clicks>0:
        print("You will not be able to run some of the remaining parts of this notebook")

In [ ]:
# required imports from lmfit
from lmfit import Model, Parameters

# import model from QENS library
import QENSmodels

[Top](#Table-of-Contents)

## Plot of the fitting model

The widget below shows the lorentzian peak shape function with a constant background imported from QENSmodels where the functions' parameters *Scale*, *Center*, *FWHM* and *background* can be varied.

In [ ]:
ini_values = {'scale': 5, 'center': 5, 'hwhm': 3, 'background':0}

def mplplot(df, **kwargs):
    fig = df.plot(legend=False).get_figure()
    plt.grid()
    plt.ylabel('lorentzian')
    plt.xlabel('x')
    plt.close(fig)
    return fig

def lorentz(scale=1.0, center=1.0, hwhm=3.0, background=0.0, view_fn=mplplot):
    xs = np.linspace(-10,10,100)
    ys = QENSmodels.lorentzian(xs, scale, center, hwhm) + QENSmodels.background_polynomials(xs, background)
    df = DataFrame(dict(y=ys), index=xs)
    return view_fn(df, scale=scale, center=center, hwhm=hwhm, background=background)

scale  = pnw.FloatSlider(name='scale', value=ini_values['scale'], start=1., end=10)
center = pnw.FloatSlider(name='center', value=ini_values['center'], start=0, end=10)
hwhm = pnw.FloatSlider(name='hwhm', value=ini_values['hwhm'], start=1, end=5)
background = pnw.FloatSlider(name='background', value=ini_values['background'], start=-1, end=1)

def on_click(event):
    """Reset the interactive plots to inital values."""
    print("reset values")
    scale.value=ini_values['scale']
    center.value=ini_values['center']
    hwhm.value=ini_values['hwhm']
    background.value=ini_values['background']
    
reset_button = pnw.Button(name='Reset')

widgets    = pn.Column("#### Lorentzian", scale, center, hwhm, background, reset_button)
lorentz_panel = pn.Row(lorentz(scale.value, center.value, hwhm.value, background.value), widgets)

def update(event):
    lorentz_panel[0] = lorentz(scale.value, center.value, hwhm.value, background.value)
    
scale.param.watch(update, 'value')
center.param.watch(update, 'value')
hwhm.param.watch(update, 'value')
background.param.watch(update, 'value')

reset_button.param.watch(on_click, 'clicks')

lorentz_panel

[Top](#Table-of-Contents)

## Creating the reference data

In [ ]:
# Create array of reference data: noisy lorentzian with background
xx = np.linspace(-5, 5, 100)
lorentzian_noisy = QENSmodels.lorentzian(xx, 0.89, -0.025, 0.45)*(1+0.1*np.random.normal(0,1,100)) + 0.5+ 0.01*np.random.normal(0,1,100)

[Top](#Table-of-Contents)

## Setting and fitting

In [ ]:
def flat_background(x, A0):
    """ Define flat background ot be added to fitting model"""
    return QENSmodels.background_polynomials(x, A0)

In [ ]:
gmodel = Model(QENSmodels.lorentzian) + Model(flat_background)
print('Names of parameters:', gmodel.param_names)
print('Independent variable(s):', gmodel.independent_vars)

initial_parameters_values = [1, 0.2, 0.5, 0.33]

# Fit
result = gmodel.fit(lorentzian_noisy, x=xx, 
                    scale=initial_parameters_values[0], 
                    center=initial_parameters_values[1],
                    hwhm=initial_parameters_values[2],
                    A0=initial_parameters_values[3])

In [ ]:
# Plot initial model and reference data
fig0, ax0 = plt.subplots()
ax0.plot(xx, lorentzian_noisy, 'b-', label='reference data')
ax0.plot(xx, result.init_fit, 'k--', label='model with initial guesses')
ax0.set(xlabel='x', title='Initial model and reference data')
ax0.grid()
ax0.legend();

[Top](#Table-of-Contents)

## Plotting results

using methods implemented in `lmfit`

In [ ]:
# display result
print('Result of fit:\n',result.fit_report())

# plot fitting result using lmfit functionality
result.plot()

In [ ]:
# Plot fitting result and reference data using matplotlib.pyplot
fig1, ax1 = plt.subplots()
ax1.plot(xx, lorentzian_noisy, 'b-', label='reference data')
ax1.plot(xx, result.best_fit, 'r.', label='fitting result')
ax1.legend()
ax1.set(xlabel='x', title='Fit result and reference data')
ax1.grid();

In [ ]:
for item in ['hwhm', 'center', 'scale']:
    print(item, result.params[item].value, '+/-', result.params[item].stderr, dict_physical_units[item])